In [1]:
from helper import print_sents, accuracy, tokenized_word_sets

**tf.tsv**

always predict A-coref as True and B-coref as False

In [2]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [3]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


**closer.tsv**

Predict closer coreference to pronoun as True

In [4]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [5]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

Overall recall: 50.1 precision: 44.4 f1: 47.1gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94


  with open(filename, 'rU') as f:


In [6]:
print_sents(random=True)

Buck and Annie had been praying for her younger siblings for quite a while, so for the two children to come live with the couple was ``a foregone conclusion.`` There was some conflict as the two families tried to merge. Soon after Nelly and Matthew came to live with the Herrings, the three siblings began singing together. Annie usually sang lead, Matthew harmonized, and Nelly's rich alto voice found the missing notes between the two. Although she rarely sang lead, Greisen's harmonies tended to be more complicated than her brother's.

First George Holley beat both amateur wing halves to open the scoring, the second came from Harold Fleming's ``splendid left-foot shot'', and Harry Hampton converted a corner. The half finished 4--0, after Hampton converted a penalty awarded against Arthur Knight for handball when Fleming's powerful shot struck his arm.

Judy Pace appeared in film and TV, appearing in the TV show The Young Lawyers and the film Frogs, and Cleavon Little made nightclub perfo

In [7]:
import pandas as pd

In [8]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [9]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [10]:
result = open("subject.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [11]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject.tsv

Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


In [12]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# subject.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [13]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both word coreference is false
4. When the pronoun is subject pronoun

 * When only one word is subject
 * When both words are subject
 * When both words are not subject

5. When the pronoun is object/possessive pronoun
 * When the pronoun and words are in different sentence
 * When the pronoun and words are in same sentence
 * When only one word is in same sentence with the pronoun
    

In [14]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [15]:
parenthesis_data = []

def isparenthesis(sent, word, offset):
    if sent[offset-1] == "(" and sent[offset+len(word)] == ")":
        return True
    else:
        return False

for row in train_data.iloc:
    if isparenthesis(row['Text'], row['A'], row['A-offset']) or \
       isparenthesis(row['Text'], row['B'], row['B-offset']):
            parenthesis_data.append(row)

In [16]:
parenthesis_data = pd.DataFrame(parenthesis_data, columns=train_data.columns)

In [17]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

Once believed lost, The Shakedown was discovered and restored by the staff of the George Eastman House in 1998. Director William Wyler made the move up to talking pictures with this blend of action and comedy. Dave Roberts (James Murray) is a professional boxer better at losing in fixed matches than in knocking out his opponents.

Dugan, he humorously blows on his nose and lets him drop to the ground (in the same style that Mr. Miyagi defeated Kreese at the start of the second movie) where his students look down at him with disappointing looks. Louisa Pierce (Constance Towers) Julie's grandmother and only known living family member. She and Mr.

Rinkie is about to get married to Vicky (Sharad Kapoor) but she is unaware of his drug dealings. Raju gives Rajesh eight days to break the news to his family and call Raju his son or he'll tell the family himself. After he leaves, Rajesh gets afraid, and hires gangsters to attack him. On his way out, Raju gets beat up by the hired gangsters, th

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [18]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [19]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [20]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

Crime boss Rusty Pirone (Armand Assante) is about to stand trial again and Daniel Graham (Gabriel Byrne) of the district attorney's office is determined this time to put him behind bars. Pirone sends one of his henchmen, a burned-out ex-cop named Tommy Vesey (William Hurt), to threaten Valerie Alston (Joanne Whalley-Kilmer), a juror. Unless she cooperates by finding Pirone innocent and deadlocking the jury, Vesey intends to do harm to Valerie's young son.

Livsey eventually cracks under the pressure and insists that she will reveal the truth to anyone other than Jasper Pine. Archie convinces her to accompany him to the brownstone for her protection, where Wolfe summons Cecily Pine by informing her that he knows who the murderer is. When she arrives, Cecily Pine confirms Wolfe's suspicions--the murderer was her husband, Jasper Pine.

The album's front cover features artwork by Glasgow writer and artist Alasdair Gray. Guitarist Chris Connick notes that: Our friend Sorcha Dallas (and wife

In [21]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [22]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-21-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-21-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [23]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [24]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [25]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [26]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

However, Arjun manages to escape and is rescued by Guru Bahula,the royal priest of Bali. Bahula manages to escape with Arjun and hide in a cave. Arjun is found unconscious lying on a bed. He soon wakes up and addresses Bheem and friends as mere kids. Chutki and Raju feel that they shouldn't help Arjun at all. Bheem explains his friends to keep patience and asks them not to leave Arjun in this dangerous situation.

He drew obsessively from childhood obsessed, both from his own imagination and by copying images that particularly pleased him. 1913 after his return from Ascona, he attended painting school in Munich for a total of eight days. But doubt in his ability was so strong that he hid his work from all but his friend, the writer Oskar Maria Graf (as Schrimpf former baker).

She shifted to high school, working in Gu*nica between 1942 and 1957. In 1947 and again in 1955, she served as acting school principal. Her first marriage ended in divorce and on December 25, 1944, she married Is

## 4. When the pronoun is subject pronoun

### * And when only one word is subject

In [61]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
other_data = train_data[train_data['Pronoun'].isin(['his', 'hin', 'her', 'His', 'Him', 'Her'])]
print(len(subject_data))
print(len(other_data))

801
1101


In [28]:
print_sents(datas={'manual':True, 'data':subject_data}, num_lines = 10)

By the end of the series, Jeri snaps out her depression, thanks to Takato and Calumon, and realizes that humans can make their own destiny and that she was never meant to be alone. It was never stated but certainly Implied that Calumon was her second partner. Jeri not only attended Rika's birthday, but she along with Calumon also helped Rika's grandmother, Seiko Hata with all the decorations as well.

Ftorek was one of the top scorers in the league but Messier managed to get only two goals. He would play 47 games for the Stingers tallying only one goal and ten assists. While in Cincinnati, Messier was teammates with Mike Gartner, Barry Melrose and Paul Stewart. When he retired, Messier was the last former WHA player still active on the ice as a player.

Like many other religious houses, they profited by the embarrassment of lesser barons and knights, and in 1232 were able to purchase the greater part of the manor of Alvingham from John de Melsa, his father and mother, by paying off the

In [29]:
subject_data.iloc[204]

ID                                                  development-509
Text              At his trial Jonah Anguka had based his defenc...
Pronoun                                                          he
Pronoun-offset                                                  583
A                                                             Cohen
A-offset                                                        479
A-coref                                                       False
B                                                            Anguka
B-offset                                                        510
B-coref                                                        True
URL                       http://en.wikipedia.org/wiki/Jonah_Anguka
Name: 508, dtype: object

In [30]:
print(subject_data.iloc[203]['Text'])
words, tagged_words, word_index = tokenized_word_sets(row)
print(tagged_words)

By the 1970s Bergman had already received two Academy Awards from five nominations, but went on to be nominated twice more, winning for a third time, this time in the category of Best Supporting Actress, for 1974's Murder on the Orient Express, based on the Agatha Christie novel of the same name, for which she also received her first and only BAFTA.
[('Watkins', 'NNS'), ('was', 'VBD'), ('a', 'DT'), ('close', 'JJ'), ('friend', 'NN'), ('of', 'IN'), ('Hess', 'NNP'), ("'", 'POS'), ('first', 'JJ'), ('wife', 'NN'), (',', ','), ('Katherine', 'NNP'), ('Hawley', 'NNP'), ('.', '.'), ('She', 'PRP'), ('was', 'VBD'), ('one', 'CD'), ('of', 'IN'), ('the', 'DT'), ('first', 'JJ'), ('to', 'TO'), ('send', 'VB'), ('sympathy', 'NN'), ('when', 'WRB'), ('Hawley', 'NNP'), ('was', 'VBD'), ('thrown', 'VBN'), ('from', 'IN'), ('a', 'DT'), ('horse', 'NN'), ('and', 'CC'), ('killed', 'VBN'), ('.', '.'), ('Watkins', 'NNS'), ('and', 'CC'), ('Hess', 'NNP'), ('had', 'VBD'), ('a', 'DT'), ('son', 'NN'), (',', ','), ('Adam

In [50]:
def word_subject(row):
    raw = row['Text']
    words, tagged_words, word_index = tokenized_word_sets(row)

    A_id = row['A-offset']
    B_id = row['B-offset']

    def next_word(word):

        tokens = len(row[word].split())
        id = row[word+'-offset']
        index = word_index[id] + tokens

        if tagged_words[index] == ('(', '('):
            close = tagged_words[index:].index((')', ')'))
            if index + close + 1 >= len(tagged_words):
                return tagged_words[index]
            return tagged_words[index + close + 1]
        
        return tagged_words[index]

    if next_word('A')[1].startswith('V'):
        if next_word('B')[1].startswith('V'):
            return (True, True)
        else:
            return (True, False)
    else:
        if next_word('B')[1].startswith('V'):
            return (False, True)
        else:
            return (False, False)

def only_subject(row):
    return sum(word_subject(row))%2 == 1

In [51]:
only_subject_data = subject_data.apply(only_subject, axis=1)
only_subject_data = subject_data[only_subject_data]
len(only_subject_data)

462

In [52]:
only_subject_data['A-pred'] = only_subject_data.apply(lambda row: word_subject(row)[0], axis=1)
only_subject_data['B-pred'] = only_subject_data.apply(lambda row: word_subject(row)[1], axis=1)

<ipython-input-52-146d5bf28ab4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_subject_data['A-pred'] = only_subject_data.apply(lambda row: word_subject(row)[0], axis=1)
<ipython-input-52-146d5bf28ab4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_subject_data['B-pred'] = only_subject_data.apply(lambda row: word_subject(row)[1], axis=1)


In [53]:
accuracy(only_subject_data)

0.8246753246753247

### * And when both words are subject

In [54]:
def both_subject(row):
    return word_subject(row) == (True, True)

In [55]:
both_subject_data = subject_data.apply(both_subject, axis=1)
both_subject_data = subject_data[both_subject_data]
len(both_subject_data)

54

In [56]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 30)

After it becomes clear to her that Judah will not end his marriage, Dolores threatens to disclose the affair to Judah's wife, Miriam (Claire Bloom). Dolores's letter to Miriam is intercepted and destroyed by Judah, but Dolores sustains the pressure on him with threats of revelation. She is also aware of some questionable financial deals Judah has made, which adds to his stress.

The two reconcile after that, but a Negative Zone creature attacks them. They manage to defeat the creature, but Lyja passes out from her injuries. A bit later, when the ``new'' Fantastic Four fly off to the prison, Franklin and Valeria are grabbed by Negative Zone creatures, but Lyja saves them. She tells Johnny that she was working in a book store when a Skrull approached her, telling her to join them.

He came back every year to Le Mans, fourteen times in total, where he won in 1968 with Belgian driver Lucien Bianchi in a Ford GT40 of the Wyer -- Gulf team. In 1962, Ricardo died in a horrific accident while 

Assume when both words are subject, closer word is True, the other is False
--> accuracy = 22/30 = 73.3%

### * And when both words are not subject

In [57]:
def none_subject(row):
    return word_subject(row) == (False, False)

In [58]:
none_subject_data = subject_data.apply(none_subject, axis=1)
none_subject_data = subject_data[none_subject_data]
len(none_subject_data)

285

In [59]:
print_sents(datas={'manual':True, 'data':none_subject_data}, num_lines = 10)

Lapid's spy-themed literature is especially popular in Israel. She is the creator of the character Lizzy Bdihi, who is featured in six of Lapid's thriller novels. Bdihi is a journalist at a Beersheba Local Newspaper called Hazman Darom (literally ``The Southern Times''), and in each book she stubbornly insists on dealing with a detective mystery no one asked her to solve.

She hosted Lancaster, California radio station KVVS's show, Maximum Rotation. She has performed all over the world, including, Detroit, Michigan, Cambridge, Singapore, Calgary, Hollywood, Montreal, and Omaha. She has been compared to Kylie Minogue and Deborah Harry. She's also been called Om Records' First Lady of House.

Throsby's third album, ``A Loud Call'', was recorded in Nashville by Mark Nevers (Lambchop, Andrew Bird) with string and horn arrangements recorded in the Kangaroo Valley, NSW, by Dupe. Featuring guest vocals by Will Oldham a.k.a. Bonnie 'Prince' Billy and guest musicians fromLambchop and Silver Jew

## 5. When the pronoun is object/possessive pronoun

### * And when all words are in different sentence with the pronoun

In [60]:
def different_sents(row):
    raw = row['Text']
    words, tagged_words, word_index = tokenized_word_sets(row)

    dots = [i for i, w in enumerate(words) if w == '.']

    pro_id = word_index[row['Pronoun-offset']]
    A_id = word_index[row['A-offset']]
    B_id = word_index[row['B-offset']]

    def between(id1, id2):
        if id1 > id2:
            id1, id2 = id2, id1
        dots = [i for i, w in enumerate(words[id1:id2]) if w == '.']
        if len(dots) > 0:
            return True
        else:
            return False

    return (between(pro_id, A_id), between(pro_id, B_id))

def both_different_sents(row):
    return different_sents(row) == (True, True)
    

In [63]:
both_different = other_data.apply(both_different_sents, axis=1)
both_different = other_data[both_different]
len(both_different)

205

In [64]:
print_sents(datas={'manual':True, 'data':both_different}, num_lines = 10)

Louisa Yeomans was born on 17 October 1863 in Washington, New Jersey, the third of five children of Alfred and Elizabeth Blythe (Ramsay) Yeomans. Her father was a Presbyterian minister.

She played as one of the children of Carding (played by Ronald Corveau) and Luisa (played by Marianne dela Riva) of Gulong Ng Palad on television. She became a famous child star of the '80s along with Lady Lee, RR Herrera, Chuckie Dreyfus, Jaypee de Guzman, Rose Ann Gonzales, Glaiza Herradura, Matet de Leon, and Aiza Seguerra, among others. Her other projects were Barbi: Maid In The Philippines (1989) with Joey de Leon and Petrang Kabayo And Pilyang Kuting (1990) with Roderick Paulate, among others.

He became head of the Department of Anthropology at Mexico City College, which position he held at the time of his passing on January 2, 1951. According to fellow anthropologist Charles E. Dibble, ``In the brief span of a decade, Barlow gave Middle American research an impetus and perspective of enduring c